In [1]:
# Folder setup
from google.colab import drive
drive.mount('/content/drive')
!ln -s "/content/drive/My Drive/MLGO" "/content/MLGO"

import os
os.chdir("/content/MLGO")

Mounted at /content/drive


In [3]:
!git config --global core.fileMode false
!git config --global user.email "colab@google"
!git config --global user.name "Colab"

In [ ]:
# Install packages

In [31]:
# Imports for ML
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import tensorflow.keras as keras
import tensorflow as tf

# Other third-party imports
import pandas as pd
import numpy as np
import yaml

# Imports from standard library
import glob

# Read config
with open("config/config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [24]:
# Load data
with np.load(f"{config['path']['final']}/labels.npz", allow_pickle=True) as D:
    labels = D['labels']
    features = D['features']
    samples = D['samples']

with np.load(f"{config['path']['final']}/train.npz") as D:
    X_train, Y_train = D['X'], D['Y']

with np.load(f"{config['path']['final']}/test.npz") as D:
    X_test, Y_test = D['X'], D['Y']

with np.load(f"{config['path']['final']}/val.npz") as D:
    X_val, Y_val = D['X'], D['Y']

# Model Development

## Autoencoder

In this section, I will develop an autoencoder model for reconstructing log fold-change values

### Model Parameters

* `ENCODING_DIM` - The encoding dimension is a hyperparameter that determines the size of the bottleneck layer in the autoencoder. It represents the number of neurons in the hidden layer that encodes the input data into a lower dimensional representation.

In [34]:
# Parameters
INPUT_SHAPE = (X_train.shape[1],)
ENCODING_DIM = 32
BATCH_SIZE = 256
EPOCHS = 50

In [33]:
# Model Architecture

## Encoder
encoder = Sequential([
    Dense(ENCODING_DIM, activation='relu', input_shape=INPUT_SHAPE)
])

## Decoder
decoder = Sequential([
    Dense(INPUT_SHAPE[0], activation='relu')
])

## Autoencoder
autoencoder = Sequential(encoder.layers + decoder.layers)

## Compile
autoencoder.compile(
    optimizer='adam',
    loss='mean_squared_error'
)

In [ ]:
# Train the network
history = autoencoder.fit(
    X_train,
    X_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True,
    validation_data=(X_val, X_val)
)

Epoch 1/50
18/18 [==============================] - 10s 520ms/step - loss: 210132016.0000 - val_loss: 79769488.0000
Epoch 2/50
18/18 [==============================] - 7s 374ms/step - loss: 209660944.0000 - val_loss: 79782944.0000
Epoch 3/50
18/18 [==============================] - 9s 523ms/step - loss: 208454752.0000 - val_loss: 79778912.0000
Epoch 4/50
18/18 [==============================] - 7s 377ms/step - loss: 207016064.0000 - val_loss: 79779000.0000
Epoch 5/50
18/18 [==============================] - 9s 531ms/step - loss: 205468864.0000 - val_loss: 79781072.0000
Epoch 6/50
18/18 [==============================] - 7s 375ms/step - loss: 203392912.0000 - val_loss: 79782832.0000
Epoch 7/50
18/18 [==============================] - 9s 524ms/step - loss: 200591184.0000 - val_loss: 79788712.0000
Epoch 8/50
18/18 [==============================] - 7s 372ms/step - loss: 197586880.0000 - val_loss: 79792592.0000
Epoch 9/50
18/18 [==============================] - 9s 538ms/step - loss: 19518